In [1]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv
# new
from openai import AsyncOpenAI
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# Load the environment variables from .env file
load_dotenv()

# Retrieve the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

# Configuration
openai.api_key = openai_api_key
model_name = 'gpt-4o'  # Ensure this model version supports sessions

prompt = ('You are a classifier. In the following, I am going to provide figures from different peer-reviewed papers, and I want you to classify them according to the following '
              + 'categories each time an new image is passed: \n\nCategories: '+ 
              '\"process diagram\", \"2x2 matrix\", \"venn diagram\", \"conceptual diagram\", \"cycle\", \"hierarchical diagram\", \"bar chart\", ' + 
              '\"stacked bar chart\", \"line graph\", \"scatter plot\", \"mixed statistical plot (more than 1 statistical plot type)\", \"data structure\",' + 
              ' \"data collection, data analysis, data gathering diagrams\", \"heatmap\", \"data map\", \"organizational chart\", \"timeline\", \"drawing\", \"photo\"' + 
              '\n\nPlease choose the category that best represents and most specifically describes the image. Your response should only be an exact string matching one of the categories. '
              )

client = AsyncOpenAI()

# Load the Excel file into a DataFrame
cropped_and_labeled_df = pd.read_excel("../cropped_and_labeled_figs.xlsx")

count = 0
# Loop through rows and check for 'none selected'
for index, row in cropped_and_labeled_df.iterrows():
    if row["subcategory"] == "none selected":
        base64_image = encode_image("../" + row["new image path"])
        print(row["new image path"])
        image_completion = await client.chat.completions.create(model="gpt-4o", messages=[{"role": "user","content": [    
            {
                "type": "text", 
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                },
                
            }]}])
        subcategory = image_completion.choices[0].message.content.replace('"', '')
        print(subcategory)
        cropped_and_labeled_df.iloc[index, cropped_and_labeled_df.columns.get_loc('subcategory')] = subcategory
        count = count + 1

print(f'Images classified: {count}')
cropped_and_labeled_df.to_excel("../cropped_and_labeled_figs.xlsx", index=False)

final_figures/2022/F2_P9_Kellogg_2022_OrgSci_Local Adaptation without Work Intensification.png
conceptual diagram
final_figures/2022/F1_P8_Kellogg_2022_OrgSci_Local Adaptation without Work Intensification.png
cycle
final_figures/2022/F4_P33_Sawyer & Clair_ASQ_2022_Hope Cultures in Organizations - Tackling the Grand Challenge of Commercial Sex Exploitation.png
cycle
final_figures/2022/F3_P30_Sawyer & Clair_ASQ_2022_Hope Cultures in Organizations - Tackling the Grand Challenge of Commercial Sex Exploitation.png
conceptual diagram
final_figures/2022/F1_P5_Hedberg & Lounsbury_2022_OrgSci_Not Just Small Potatoes.png
mixed statistical plot (more than 1 statistical plot type)
final_figures/2022/F2_P9_Waldorff & Madsen_2022_OrgStudies_Translating to Maintain Existing Practices.png
conceptual diagram
final_figures/2022/F3_P11_Lingo_2022_OrgStudies_Digital Curation and Creative Brokering - Managing information overload in open organizing.png
conceptual diagram
final_figures/2022/F2_P17_Chapple, 